# Quantum Demonstration Playground

This notebook will be used for demonstrations of simple gate computations. Import statements and some basic stubs are provided.

In [ ]:
import numpy as np
import matplotlib as plt
%matplotlib inline
from IPython.display import Latex

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, Aer, transpile
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import Statevector, Operator

# Demonstrations

First we create a simple state vector that represents a single qubit, and initialize it to the basis state `|0>`

In [ ]:
q = np.array([[1], [0]])
q

## Gate Operations

### Simple Gates

Below, we define the following 1 qubit gates:

* The identity gate $I = \begin{pmatrix}1&0\\0&1\end{pmatrix}$
* The Pauli-X gate (X/NOT) $X = \begin{pmatrix}0&1\\1&0\end{pmatrix}$
* The Hadamard gate $H = \frac{1}{\sqrt{2}}\begin{pmatrix}1&1\\1&-1\end{pmatrix}$

As well as the 2-qubit CNOT gate:
$$
CNOT =
\begin{pmatrix}
    1 & 0 & 0 & 0\\
    0 & 1 & 0 & 0\\
    0 & 0 & 0 & 1\\
    0 & 0 & 1 & 0
\end{pmatrix}
$$

In [ ]:
# Gate definitions
i = np.array([[1, 0], [0, 1]])
x = np.array([[0, 1], [1, 0]])
h = (1 / np.sqrt(2)) * np.array([[1, 1], [1, -1]])
cnot = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]])

i, x, h, cnot

We will also leverage `qiskit` to visualize simple circuits using these gates and their corresponding state vectors.

First, our state vector in Dirac notation and visualized on the Bloch sphere, as well as our circuit (with a gate of choice):

In [ ]:
state = Statevector(q)
state.draw('latex')

In [ ]:
state.draw('bloch')

In [ ]:
circuit = QuantumCircuit(1)

# circuit.i(0)
# circuit.x(0)
circuit.h(0)

circuit.draw('mpl')

Now we "evolve" the state by running the circuit and look at the state (note that we are not "measuring" the state, but simply computing what the state is expected to be):

In [ ]:
state = state.evolve(circuit)
state.draw('latex')

In [ ]:
state.draw('bloch')

### Computing Hadamard Gate on Basis States

Let's run through the example math manually of applying the Hadamard gate to the basis state `|0>`:

$$
\begin{align}
|\psi\rangle &= H\begin{bmatrix}1\\0\end{bmatrix}\\
&= \frac{1}{\sqrt{2}}
\begin{pmatrix}
    1 & 1\\
    1 & -1
\end{pmatrix}
\begin{bmatrix}1\\0\end{bmatrix}\\
&=
\begin{bmatrix}
    \frac{1}{\sqrt{2}}\\
    \frac{1}{\sqrt{2}}
\end{bmatrix}
\end{align}
$$

In [ ]:
q

In [ ]:
(h @ q)

We can determine that the probabiliy of measuring each basis state is 50% by raising the state vector to the power of 2:

$$
\begin{align}
P &=
\begin{bmatrix}
    \frac{1}{\sqrt{2}}\\
    \frac{1}{\sqrt{2}}
\end{bmatrix}^2\\
&=
\begin{bmatrix}
    \frac{1}{2}\\
    \frac{1}{2}
\end{bmatrix}
\end{align}
$$

In [ ]:
(h @ q) ** 2

This indicates that the qubit is in a superposition, where a measurement will result in `|0>` half of the time, and `|1>` the other half.

## Deutsch Problem

The Deutsch Problem is a simple "toy" example of how a problem that takes exponential time in classical computing takes only polynomial time when solved (in a fundamentally different way) with quantum computing. For this example, we'll use $f(x) = x$ as our "balanced" function. For its unitary matrix representation $U_f$, we can represent this function with the CNOT gate, as it satisifies the property of $U_f: |x\rangle|y\rangle \rightarrow |x\rangle|y \oplus f(x)\rangle$

The truth table for this function is as follows, including the relevant values for the ancilla qubit $y$:

| x | y | f(x) | y xor f(x)      |
|---|---|------|-----------------|
| 0 | 0 |  0   | 0               |
| 1 | 0 |  1   | 1               |
| 0 | 1 |  0   | 1               |
| 1 | 1 |  1   | 0               |



Note that [Qiskit uses little endian](https://qiskit.org/documentation/stubs/qiskit.circuit.library.CXGate.html#qiskit.circuit.library.CXGate) for its qubit ordering, so we apply our CNOT matrix with $q_1$ as the control and $q_0$ as the target here.

In [ ]:
# Showing how CNOT gate changes basis states for a 2-qubit system
result_table = []
for index in range(4):
    instate = np.array([[1 if j == index else 0] for j in range(4)])
    result = cnot @ instate
    result_table.append((instate, result))

for result_tuple in result_table:
    instate_tex = Statevector(result_tuple[0]).draw('latex_source')
    result_tex = Statevector(result_tuple[1]).draw('latex_source')
    display(Latex('$CNOT \otimes {} = {}$'.format(instate_tex, result_tex)))

In [ ]:
qr = QuantumRegister(2, 'q')
cr = ClassicalRegister(1, 'c')
circuit = QuantumCircuit(qr, cr)
circuit.x(1)
circuit.h(0)
circuit.h(1)
# circuit.cnot(0, 1)
circuit.unitary(cnot, (1, 0), 'CNOT')
# circuit.unitary(np.kron(i, i), (0, 1), 'Identity')
circuit.h(0)
circuit.measure(0, 0)
circuit.draw('mpl')

In [ ]:
sim_backend = Aer.get_backend('qasm_simulator')

job = sim_backend.run(transpile(circuit, sim_backend), shots=1024)
result = job.result()

counts = result.get_counts()

plot_histogram(counts)